In [ ]:
import os
from scipy.io import loadmat
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# the following import is required for matplotlib < 3.2:
from mpl_toolkits.mplot3d import Axes3D  # noqa
import mne


In [ ]:
# load raw
subject_id = 2
before_or_after = 'after'
raw_path = os.path.join('..', '..', 'data', str(subject_id), 'raw_' + before_or_after + '.fif')
raw = mne.io.read_raw_fif(raw_path, preload=True)
# raw.plot()

In [ ]:
# check psd
psd_obj = raw.compute_psd(fmax=100)
mne.viz.plot_raw_psd(raw, fmax=100, spatial_colors=True)

In [ ]:
# # filtering and resampling
# # Apply low-pass filter
# raw_filtered = raw.copy().filter(l_freq=None, h_freq=50)

# # Resample the data
# sfreq = 600
# raw_resampled = raw_filtered.copy().resample(sfreq=sfreq)

# # Plot the power spectral density (PSD) of the original data
# psd_obj = raw.compute_psd(fmax=100)
# mne.viz.plot_raw_psd(raw, fmax=100, spatial_colors=True)

# # Plot the PSD of the resampled data
# raw_resampled.plot_psd()
# psd_obj = raw_resampled.compute_psd(fmax=100)
# mne.viz.plot_raw_psd(raw_resampled, fmax=100, spatial_colors=True)

In [ ]:
# ica
raw_filtered = raw.copy() # raw already filtered
n_components = 32  # Number of ICA components (adjust as needed)
ica = mne.preprocessing.ICA(n_components=n_components, random_state=97, method='picard')
ica.fit(raw_filtered)
# Extract explained variance of all components
explained_var = ica.get_explained_variance_ratio(raw_filtered)
print(explained_var)
ica.plot_components()
ica.plot_sources(raw_filtered)

In [ ]:
# plot properties based on epochs
# Find events related to the 'fixation' stim channel
stim_channel_name = 'fixation'
event_id = {'fixation': 1}
events = mne.find_events(raw, stim_channel=stim_channel_name, min_duration=1/raw.info['sfreq'])

# Create epochs
tmin, tmax = -1.5, 3.58  # Define the time range of epochs 3.583s-5.05s
epochs = mne.Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=(0, 0), preload=True)

# Apply ICA to the epochs
ica_epochs = ica.apply(epochs.copy(), exclude=ica.exclude)

# Plot properties for each component using segments based on epochs
for idx in range(ica.n_components_):
    ica.plot_properties(ica_epochs, picks=idx)

In [ ]:
# plot overlay
# ica.plot_overlay(raw, exclude=[1,2,3,4], picks='eeg')

In [ ]:
# # Load the CSV file
# csv_file = 'ic_to_remove.csv'
# df = pd.read_csv(csv_file)

# # Get the remove_ic
# selected_row = df.loc[(df['subject_id'] == subject_id) & (df['before_or_after'] == before_or_after)]
# remove_ic = selected_row['remove_ic'].iloc[0].split(',')

# # # Get the other
# # other = selected_row['other'].iloc[0].split(',')

# # # Combine the two lists
# # remove_ic = remove_ic + other

# # Remove the components
# ica.exclude = [int(i) for i in remove_ic]
# raw_ica_applied = raw.copy()
# ica.apply(raw_ica_applied)

# # Plot the raw and ICA-applied data
# raw.plot()
# raw_ica_applied.plot()